In [118]:
from bokeh.io import output_notebook, show
from bokeh.models import CustomJS, ColumnDataSource, Slider, CDSView, IndexFilter
from bokeh.models.widgets import Select
from bokeh.layouts import column, row, layout, widgetbox
from bokeh.plotting import figure
from bkcharts import Line, Scatter
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN

#Set output to notebook
output_notebook()

df = pd.read_csv("data.csv")

columns = df.columns.values
original_df = ColumnDataSource(df)

#Create clusters
kmeans = KMeans(n_clusters=3).fit(df.as_matrix())
labels_km = kmeans.labels_

ac = AgglomerativeClustering(n_clusters=3).fit(df.as_matrix())
labels_ac = ac.labels_

df = df.assign(labels_ac = labels_ac)
df = df.assign(labels_km = labels_km)

colors = {0:'red', 1:'blue', 2:'green', 3:'orange', 4:'brown'}

df["colors_ac"] = df["labels_ac"].map(colors)
df["colors_km"] = df["labels_km"].map(colors)

#Setup bokeh widgets and plots
source = ColumnDataSource(data=dict(x=df.loc[:, 'Fresh'], 
                                    y=df.loc[:, 'Milk'], 
                                    colors_km = df.loc[:, 'colors_km'], 
                                    colors_ac = df.loc[:, 'colors_ac']))

select_x = Select(title='Select X axis:',
                  value='Fresh',
                  width=200,
                  options=columns.tolist())

select_y = Select(title='Select Y axis:',
                  value="Milk",
                  width=200,
                  options=columns.tolist())

fig1=figure(title = "KMeans")
fig1.circle('x', 'y', fill_color='colors_km', source=source, size = 10)

fig2=figure(title = "Hierarchical")
fig2.circle('x', 'y', fill_color='colors_ac', source=source, size = 10)

#Establish callbacks
update_x = CustomJS(args=dict(original_df=original_df, select_x=select_x, source=source), code="""
    var data = original_df.get('data')
    var data_source = source.get('data')
    x_attr = select_x.value
    data = data[x_attr]
    x = data_source['x']
    for(i = 0; i < x.length; i++){
        x[i] = data[i]
    }
    source.change.emit()
""")

select_x.js_on_change('value', update_x)

update_y = CustomJS(args=dict(original_df=original_df, select_y=select_y, source=source), code="""
    var data = original_df.get('data')
    var data_source = source.get('data')
    y_attr = select_y.value
    data = data[y_attr]
    y = data_source['y']
    for(i = 0; i < y.length; i++){
        y[i] = data[i]
    }
    source.change.emit()
""")

select_y.js_on_change('value', update_y)

#Display plots
layout = layout([row(fig1, fig2), widgetbox(select_x, select_y)])

show(layout)

Loading BokehJS ...